In [1]:
import json
!pip install --upgrade pip
!pip install --upgrade transformers
!pip install --upgrade tokenizers
!pip install --upgrade torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, BertLMHeadModel,BertTokenizerFast
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 245, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 444, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [5]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


No GPU available, using the CPU instead.


In [ ]:

def preprocess_intents_json(intents_file):
    with open(intents_file, "r") as f:
        data = json.load(f)
    
    preprocessed_data = []
    
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            preprocessed_data.append(f"User: {pattern}\n")
            for response in intent["responses"]:
                preprocessed_data.append(f"Assistant: {response}\n")
    
    return "".join(preprocessed_data)


def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)


intents_file = "TUSHARKHETE_intents.json"
output_file = "training_data.txt"


preprocessed_data = preprocess_intents_json(intents_file)
save_preprocessed_data(preprocessed_data, output_file)

## Fine Tune the model with my data.
- Takes around 28 minutes with smaller gpt2 version: distilgpt2
- Saves to 'output' file
- Can be very finnicky with torch environment
- Ensure all packages dependencies with transformers are up to date, torch, torch audio etc are up to date.

## Validation data ready

In [3]:
import numpy as np


# Read the entire dataset into a list
with open('training_data.txt', 'r') as f:
    data = f.readlines()


# Shuffle the dataset
np.random.seed(1)
np.random.shuffle(data)


# Split the dataset into training and validation sets (80% - 20%)
split_index = int(len(data) * 0.8)
train_data = data[:split_index]
val_data = data[split_index:]


# Save the training and validation sets as separate files
with open('train_data.txt', 'w') as f:
    f.writelines(train_data)


with open('validation_data.txt', 'w') as f:
    f.writelines(val_data)

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, EvalPrediction
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import numpy as np
from scipy.special import softmax
from sklearn.metrics import log_loss


def compute_metrics(p: EvalPrediction):
    logits = p.predictions
    labels = p.label_ids
    probabilities = softmax(logits, axis=-1)
    loss = log_loss(labels.flatten(), probabilities.reshape(-1, probabilities.shape[-1]), labels=[i for i in range(logits.shape[-1])])
    perplexity = np.exp(loss)
    return {"perplexity": perplexity}

In [25]:
model_name = 'distilgpt2'
train_file = '/content/training_data.txt'
val_file = '/content/validation_data.txt'
output_dir ="output"

# Load GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# for param in model.base_model.parameters():
#     param.requires_grad = False # Freeze the first 3 layers (encoder.block.0, encoder.block.1, encoder.block.2)

# Load training dataset
val_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=val_file,
    block_size=128)


# Load training dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=128)

# Create data collator for language modeling,
# GPT2 Does not use masked modelling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)

# Set training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_strategy='steps',
    evaluation_strategy='steps',
    metric_for_best_model='eval_loss',
    save_steps=0.1,
    eval_steps = 0.1,
    save_total_limit=2,
    load_best_model_at_end = True,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Perplexity
16,No log,3.308007,20254.895266
32,No log,3.001828,25269.778479
48,No log,2.781224,31123.324960
64,No log,2.609310,37208.306571
80,No log,2.487752,53305.017066
96,No log,2.380833,58900.174239
112,No log,2.323606,65497.179992
128,No log,2.278943,72489.331338
144,No log,2.251704,76980.774495


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=156, training_loss=2.3164868477063303, metrics={'train_runtime': 1037.8066, 'train_samples_per_second': 0.593, 'train_steps_per_second': 0.15, 'total_flos': 20087187701760.0, 'train_loss': 2.3164868477063303, 'epoch': 3.0})

In [21]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('output/tokenizer_config.json',
 'output/special_tokens_map.json',
 'output/vocab.json',
 'output/merges.txt',
 'output/added_tokens.json')

## 2. Use the model

In [11]:
def preprocess_generation(generated_response):
  #Find the position of A in assistant and add 10 characters
  assistant_index = generated_response.find("Assistant:") + 10

  # Check if 'ASSISTANT:' is found in the text and the text starts with "Assistant"
  if assistant_index != -1:
      # Remove everything before ':', also removes repetition of user input
      generated_text = generated_response[assistant_index+1:]
      # do recursively until no assistant index is found? if it keeps printing multiple

  # given that string 'User:' comes after the desired response
  user_index = generated_text.find("User:")
  if user_index != -1:
      generated_text = generated_text[:user_index]


  return generated_text


preprocess_generation('Assistant: TESTING COOL BEANS NOT SO COOL. User: Yeah i guess so')

'TESTING COOL BEANS NOT SO COOL. '

In [16]:
model_dir = "output"  # Replace with your actual directory path
prompt = "Abrasions?"

# Load the model and tokenizer from specified locations
model = GPT2LMHeadModel.from_pretrained(model_dir)
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)

import torch

In [27]:
def generate_response(model_dir, tokenizer, prompt, max_length=200):
  """
  Generates a response from the fine-tuned GPT-2 model.

  Args:
      model_dir: Path to the directory containing the fine-tuned model components.
      tokenizer: GPT2 tokenizer object.
      prompt: User prompt to start the response generation.
      max_length: Maximum length of the generated response (default 50).

  Returns:
      Generated response as a string.
  """
  # # Set model to evaluation mode
  # model.eval()

  # Encode the prompt text
  input_ids = tokenizer.encode(prompt, return_tensors="pt")
  # Assuming tokenizer.pad_token_id is set correctly
  attention_mask = torch.where(input_ids != tokenizer.pad_token_id, 1, 0)

  # Generate text using beam search
  output = model.generate(
      input_ids=input_ids,
      max_length=max_length,
      num_beams=3,  # Number of beams for beam search
      no_repeat_ngram_size=2,  # Prevent repetition of n-grams
      early_stopping=True,  # Stop generation if score doesn't improve
      temperature=0.1,
      do_sample=True,
      attention_mask= attention_mask
  )

  # Decode the generated tokens back to text
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  generated_text = preprocess_generation(generated_text)

  return generated_text

# response = generate_response(model_dir, tokenizer, prompt)

# print(f"Generated Response: {response}")

In [ ]:
def generate_printed_loop(model_dir, tokenizer):
  # Chatbot loop
  print("Type quit to stop")
  while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
      break

    generated_text = generate_response(model,tokenizer, user_input, max_length=200)
    print(f">>> Neutrino: {generated_text}")

# Start the chatbot loop
generate_printed_loop(model, tokenizer)